## Лабораторная работа №2

#### Задание 1
Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

#### Ход работы:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# Загрузка данных
df = pd.read_csv('universal_top_spotify_songs.csv')

In [2]:
# Вывод общей информации о данных
print("Общая информация о данных:")
print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("\nТипы данных по столбцам:")
print(data.dtypes)

Общая информация о данных:
Количество строк: 10000
Количество столбцов: 20

Типы данных по столбцам:
spotify_id           object
daily_rank            int64
daily_movement        int64
weekly_movement       int64
country              object
popularity            int64
is_explicit            bool
duration_ms           int64
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
dtype: object


In [9]:
df.drop(['spotify_id'], axis=1, inplace=True)
# Разделение на признаки (X) и целевую переменную (y)
X = df.drop('country', axis=1)
y = df['country']

# Разделение данных на тренировочный и тестовый наборы
# random_state=42 - гарантирует, что данные каждый раз будут одинакого разбиваться
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков (нормализация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Гиперпараметры:
   - C (регуляризация): Определяет силу регуляризации в модели SVM (Support Vector Machine / Машина опорных векторов); более низкое значение C увеличивает штраф за неправильную классификацию, что может привести к созданию простой модели с большей обобщающей способностью.
   - gamma (ядерный коэффициент): Определяет влияние одного тренировочного примера на другие; меньшие значения gamma означают более широкие гауссовы функции и приводят к более простым/прямым границам решения.
   - kernel (ядро): Определяет тип использованного ядра (линейное или RBF); линейное ядро работает линейно, тогда как RBF (радиально-базисная функция) способно моделировать сложные границы решений.

In [10]:
param_grid = {'C': [0.01, 0.1, 1], 'kernel': ['linear', 'rbf', 'sigmoid'], 'gamma': [0.1, 1, 10]}

Подбор гиперпараметров с помощью перекрестной проверки

In [11]:
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
grid.fit(X_train_scaled, y_train)

best_params = grid.best_params_
best_score = grid.best_score_

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.174 total time=   0.0s
[CV 2/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.193 total time=   0.0s
[CV 3/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.149 total time=   0.0s
[CV 4/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.188 total time=   0.0s
[CV 5/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.181 total time=   0.0s
[CV 1/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.099 total time=   0.0s
[CV 2/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.099 total time=   0.0s
[CV 3/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.106 total time=   0.0s
[CV 4/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.100 total time=   0.0s
[CV 5/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.100 total time=   0.0s


/home/dmitrii/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


[CV 1/5] END .C=0.01, gamma=0.1, kernel=sigmoid;, score=0.099 total time=   0.0s
[CV 2/5] END .C=0.01, gamma=0.1, kernel=sigmoid;, score=0.099 total time=   0.0s
[CV 3/5] END .C=0.01, gamma=0.1, kernel=sigmoid;, score=0.106 total time=   0.0s
[CV 4/5] END .C=0.01, gamma=0.1, kernel=sigmoid;, score=0.100 total time=   0.0s
[CV 5/5] END .C=0.01, gamma=0.1, kernel=sigmoid;, score=0.100 total time=   0.0s
[CV 1/5] END ....C=0.01, gamma=1, kernel=linear;, score=0.174 total time=   0.0s
[CV 2/5] END ....C=0.01, gamma=1, kernel=linear;, score=0.193 total time=   0.0s
[CV 3/5] END ....C=0.01, gamma=1, kernel=linear;, score=0.149 total time=   0.0s
[CV 4/5] END ....C=0.01, gamma=1, kernel=linear;, score=0.188 total time=   0.0s
[CV 5/5] END ....C=0.01, gamma=1, kernel=linear;, score=0.181 total time=   0.0s
[CV 1/5] END .......C=0.01, gamma=1, kernel=rbf;, score=0.099 total time=   0.0s
[CV 2/5] END .......C=0.01, gamma=1, kernel=rbf;, score=0.099 total time=   0.0s
[CV 3/5] END .......C=0.01, 

[CV 3/5] END ......C=1, gamma=1, kernel=sigmoid;, score=0.130 total time=   0.0s
[CV 4/5] END ......C=1, gamma=1, kernel=sigmoid;, score=0.125 total time=   0.0s
[CV 5/5] END ......C=1, gamma=1, kernel=sigmoid;, score=0.131 total time=   0.0s
[CV 1/5] END ......C=1, gamma=10, kernel=linear;, score=0.224 total time=   0.0s
[CV 2/5] END ......C=1, gamma=10, kernel=linear;, score=0.236 total time=   0.1s
[CV 3/5] END ......C=1, gamma=10, kernel=linear;, score=0.174 total time=   0.0s
[CV 4/5] END ......C=1, gamma=10, kernel=linear;, score=0.237 total time=   0.1s
[CV 5/5] END ......C=1, gamma=10, kernel=linear;, score=0.200 total time=   0.0s
[CV 1/5] END .........C=1, gamma=10, kernel=rbf;, score=0.093 total time=   0.0s
[CV 2/5] END .........C=1, gamma=10, kernel=rbf;, score=0.099 total time=   0.0s
[CV 3/5] END .........C=1, gamma=10, kernel=rbf;, score=0.106 total time=   0.0s
[CV 4/5] END .........C=1, gamma=10, kernel=rbf;, score=0.100 total time=   0.0s
[CV 5/5] END .........C=1, g

Вывод лучших гиперпараметров и оценка производительности модели на тестовом наборе

In [12]:
print("Лучшие гиперпараметры:", grid.best_params_)
best_svc = grid.best_estimator_
y_pred = best_svc.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

# Получение отчета о классификации с предупреждениями
report = classification_report(y_test, y_pred, zero_division=1)
print(f'Отчет о классификации:\n{report}')

Лучшие гиперпараметры: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Точность модели: 0.25870646766169153
Отчет о классификации:
              precision    recall  f1-score   support

          PK       0.20      0.25      0.22         8
          PL       1.00      0.07      0.12        15
          PT       0.33      0.29      0.31         7
          PY       0.00      0.00      1.00        10
          RO       0.67      0.60      0.63        10
          SA       0.00      0.00      1.00        11
          SE       0.00      0.00      1.00         5
          SG       0.09      0.11      0.10         9
          SK       0.33      0.20      0.25        10
          SV       0.00      0.00      1.00        17
          TH       0.43      0.33      0.38         9
          TR       0.42      0.56      0.48         9
          TW       0.40      0.17      0.24        12
          UA       0.50      0.33      0.40        12
          US       0.20      0.68      0.31        19
          UY